In [1]:
%pip install rasterio


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [8]:
!ls

TesisCode  discriminator.h5  generator.h5  images  training_1


In [15]:
import rasterio as rs
import numpy as np
from os import listdir
from os.path import isfile, join
from tensorflow.data import Dataset

def get_srtm_dataset(geotiff_dir = "TesisCode/Dataset/download2/"):
    contents = listdir(geotiff_dir)

    array_set = False
    tiff_array = None

    for file in contents:
        filepath = join(geotiff_dir, file)
        if isfile(filepath):
            dataset = rs.open(filepath)
            data = dataset.read()
            cropped_data = data[:1,:28,:28]
            if not array_set:
                tiff_array = cropped_data
                array_set = True
            else:
                tiff_array = np.concatenate((tiff_array, cropped_data), axis=0)

    return Dataset.from_tensor_slices(tiff_array)

In [13]:
import rasterio as rs
import numpy as np
import random
from os import listdir
from os.path import isfile, join
from tensorflow.data import Dataset

def get_tiff_array(geotiff_dir = "TesisCode/Dataset/download2/"):
    contents = listdir(geotiff_dir)

    array_set = False
    tiff_array = None

    crops_per_sample = 5

    for file in contents:
        filepath = join(geotiff_dir, file)
        if isfile(filepath):
            dataset = rs.open(filepath)
            data = dataset.read()
            for i in range(crops_per_sample):
                size_rows, size_cols = np.shape(data[0])
                start_rows = random.randint(0, size_rows - 28)
                start_cols = random.randint(0, size_cols - 28)
                cropped_data = data[:1,start_rows:start_rows+28,start_cols:start_cols+28]
                if not array_set:
                    tiff_array = cropped_data
                    array_set = True
                else:
                    tiff_array = np.concatenate((tiff_array, cropped_data), axis=0)

    return tiff_array

def get_srtm_dataset(geotiff_dir = "TesisCode/Dataset/download2/"):
    tiff_array = get_tiff_array(geotiff_dir)
    return Dataset.from_tensor_slicens(tiff_array)

In [31]:
import tensorflow as tf

def process_img(img):
    print(img)
    return img
    

path = "TesisCode/Dataset/download2/*.tif"
ds = tf.data.Dataset.list_files(path).map(process_img)
ds

Tensor("args_0:0", shape=(), dtype=string)


<_MapDataset element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>

In [9]:
ds = get_tiff_array()

In [12]:
np.shape(ds)

(2500, 28, 28)